In [2]:
from pubg_util import mysql, load_schema, notifier
from sphynx import sphynx, NODE_SMALL, NODE_MEDIUM, NODE_LARGE
from pyspark.sql.functions import *
import pandas as pd
import pickle

spark = sphynx.get_spark(executor_count=8, app_name='west0_ws_seg', node_spec=NODE_LARGE)

Spark cluster not assigned. creating a new one...
Node spec: 8 executors with 48G RAM each
Job Port 4049 is assigned for requested cluster
Waiting for Spark master to be available...
Spark master launched!
Creating new Spark session, name: west0_ws_seg...
Waiting for all executors ready...
All executors connected!
Complete! elapsed time: 00:00:24


In [4]:
# 각 day 30
start_date = "2022-06-13"
end_date = "2022-07-12"

ws_start_date = "2022-07-13"
ws_day_30 = "2022-08-11"

In [5]:
gcoin = load_data_mart("pc", ws_start_date, ws_day_30, "gcoin_use")
wsus_gcoin = gcoin.where(col("event_name").like("%wsus%"))
wsus_user = wsus_gcoin.select("account_id").distinct()
costume = gcoin.where(col("event_name").like("%workshop%"))
costume_user = costume.select("account_id").distinct()

In [6]:
only_costume_gcoin = costume.join(wsus_user, "account_id", "leftanti")
only_wsus_gcoin= wsus_gcoin.join(costume_user, "account_id", "leftanti")

In [7]:
costume_and_wsus_gcoin_user = wsus_gcoin.join(costume_user, "account_id").select("account_id").distinct()

In [8]:
costume_and_wsus_gcoin = costume.select("account_id", "product_id", "paid_use", "free_use", "sub_category", "event_type", "event_name", "price") \
    .unionByName(wsus_gcoin.select("account_id", "product_id", "paid_use", "free_use", "sub_category", "event_type", "event_name", "price")) \
    .join(costume_and_wsus_gcoin_user, "account_id")

## a

In [10]:
from pyspark.sql.types import * 

def chest_cnt(product_id):
    if product_id in ["itemdesc.14300001", "itemdesc.14300002", "itemdesc.13000577", "itemdesc.13000580"]:
        return 1
    elif product_id in ["itemdesc.13000578", "itemdesc.13000581"]:
        return 5
    elif product_id == "itemdesc.13000582":
        return 10
    else:
        return 0

assign_chest_cnt = udf(chest_cnt, IntegerType())
only_costume_gcoin = only_costume_gcoin.withColumn("chest_cnt", assign_chest_cnt("product_id"))

In [11]:
only_costume_gcoin_by_user = only_costume_gcoin.groupBy("account_id") \
    .agg(sum(col("free_use") + col("paid_use")).alias("total_gcoin"), sum("paid_use").alias("paid_use"), sum("chest_cnt").alias("chest_cnt"))

In [12]:
only_costume_gcoin_by_user = only_costume_gcoin_by_user.withColumn("group", \
                                         when(col("total_gcoin") <= 1000, "A") \
                                        .when((col("total_gcoin") > 1000) & (col("total_gcoin") <= 5000), "B") \
                                        .when((col("total_gcoin") > 5000) & (col("total_gcoin") <= 10000), "C") \
                                        .when((col("total_gcoin") > 10000) & (col("total_gcoin") <= 30000), "D") \
                                        .when((col("total_gcoin") > 30000), "E") \
                                     )
only_costume_gcoin_by_user.groupBy("group").agg(countDistinct("account_id").alias("user_cnt"), sum("total_gcoin").alias("total_gcoin"), sum("paid_use").alias("paid_use"), sum("chest_cnt").alias("chest_cnt")).orderBy("group").toPandas()

,group,user_cnt,total_gcoin,paid_use,chest_cnt
0,A,399792,123376410,21088135,462038
1,B,112498,233586940,120241920,886979
2,C,20887,145386420,120895095,544300
3,D,19832,333294720,293085495,1133358
4,E,7105,406997330,363742425,1065000


In [13]:
only_costume_gcoin.join(only_costume_gcoin_by_user.select("account_id", "group"), "account_id") \
    .withColumn("type", when(col("event_name").like("%workshop%"), "workshop").otherwise(lit("costume"))) \
    .groupBy("group", "type").agg(countDistinct("account_id").alias("user_cnt"), sum(col("paid_use") + col("free_use")).alias("total_use"), sum("paid_use").alias("paid_use"), sum("chest_cnt").alias("chest_cnt")) \
    .orderBy("group", "type").toPandas()

,group,type,user_cnt,total_use,paid_use,chest_cnt
0,A,workshop,399792,123376410,21088135,462038
1,B,workshop,112498,233586940,120241920,886979
2,C,workshop,20887,145386420,120895095,544300
3,D,workshop,19832,333294720,293085495,1133358
4,E,workshop,7105,406997330,363742425,1065000


In [14]:
only_costume_gcoin_by_user = only_costume_gcoin.join(only_costume_gcoin_by_user.select("account_id", "group"), "account_id") \
    .withColumn("type", when(col("event_name").like("%workshop%"), "workshop").otherwise(lit("costume"))) \
    .groupBy("type", "account_id", "group").agg(sum("paid_use").alias("paid_use"), sum(col("paid_use")+col("free_use")).alias("total_use"), sum("chest_cnt").alias("chest_cnt")).toPandas()
only_costume_gcoin_by_user.to_csv("./only_costume_gcoin_by_user3.csv", index=False)

## b

In [15]:
def wsus_cnt(price):
    if price == 1800:
        return 10
    elif price == 200:
        return 1
    else:
        return 0
    
assign_wsus_cnt = udf(wsus_cnt, IntegerType())
costume_and_wsus_gcoin = costume_and_wsus_gcoin.withColumn("wsus_cnt", assign_wsus_cnt("price"))

In [16]:
assign_chest_cnt = udf(chest_cnt, IntegerType())
costume_and_wsus_gcoin = costume_and_wsus_gcoin.withColumn("chest_cnt", assign_chest_cnt("product_id"))

In [17]:
costume_and_wsus_gcoin_by_user = costume_and_wsus_gcoin.groupBy("account_id") \
    .agg(sum("wsus_cnt").alias("wsus_cnt"), sum("chest_cnt").alias("chest_cnt"), sum("paid_use").alias("paid_use"), sum(col("paid_use") + col("free_use")).alias("total_use"))

In [18]:
costume_and_wsus_gcoin_by_user = costume_and_wsus_gcoin_by_user.withColumn("group", \
                                                                when(col("wsus_cnt") <=3, "A") \
                                                                .when((col("wsus_cnt") >= 4) & (col("wsus_cnt") <= 15), "B") \
                                                                .when((col("wsus_cnt") >= 16) & (col("wsus_cnt") <= 30), "C") \
                                                                .when((col("wsus_cnt") >= 31) & (col("wsus_cnt") <= 240), "D") \
                                                                .when((col("wsus_cnt") >= 241), "E") \
                                                                          )
costume_and_wsus_gcoin_by_user.groupBy("group").agg(countDistinct("account_id").alias("user_cnt"), sum("total_use").alias("total_use"), sum("paid_use").alias("paid_use"), sum("wsus_cnt").alias("wsus_cnt"), sum("chest_cnt").alias("chest_cnt")).orderBy("group").toPandas()

,group,user_cnt,total_use,paid_use,wsus_cnt,chest_cnt
0,A,67144,166048380,115287335,104440,494658
1,B,32671,237158080,186871625,280105,606523
2,C,12776,198151130,171728115,295391,456105
3,D,25787,939947050,833340770,2290363,1519682
4,E,4784,642499420,574390045,2406104,494869


In [19]:
costume_and_wsus_gcoin.join(costume_and_wsus_gcoin_by_user.select("account_id", "group"), "account_id") \
    .withColumn("type", when(col("event_name").like("%workshop%"), "workshop").when(col("event_name").like("%wsus%"), "wsus").otherwise(lit("costume"))) \
    .groupBy("group", "type").agg(countDistinct("account_id").alias("user_cnt"), sum(col("paid_use") + col("free_use")).alias("total_use"), sum("paid_use").alias("paid_use"), sum("wsus_cnt").alias("wsus_cnt"), sum("chest_cnt").alias("chest_cnt")) \
    .orderBy("group", "type").toPandas()

,group,type,user_cnt,total_use,paid_use,wsus_cnt,chest_cnt
0,A,workshop,67144,145160380,111651340,0,494658
1,A,wsus,67144,20888000,3635995,104440,0
2,B,workshop,32671,184273280,157741565,0,606523
3,B,wsus,32671,52884800,29130060,280105,0
4,C,workshop,12776,143927530,125966045,0,456105
5,C,wsus,12776,54223600,45762070,295391,0
6,D,workshop,25787,523996850,461679130,0,1519682
7,D,wsus,25787,415950200,371661640,2290363,0
8,E,workshop,4784,208150820,182943485,0,494869
9,E,wsus,4784,434348600,391446560,2406104,0


In [20]:
costume_and_wsus_gcoin_by_user2 = costume_and_wsus_gcoin.join(costume_and_wsus_gcoin_by_user.select("account_id", "group"), "account_id") \
    .withColumn("type", when(col("event_name").like("%workshop%"), "workshop").when(col("event_name").like("%wsus%"), "wsus").otherwise(lit("costume"))) \
    .groupBy("type", "account_id", "group").agg(sum("paid_use").alias("paid_use"), sum(col("paid_use")+col("free_use")).alias("total_use"), sum("wsus_cnt").alias("wsus_cnt"), sum("chest_cnt").alias("chest_cnt")).toPandas()
costume_and_wsus_gcoin_by_user2.to_csv("./costume_and_wsus_gcoin_by_user3.csv", index=False)

## c

In [21]:
only_wsus_gcoin = only_wsus_gcoin.withColumn("wsus_cnt", assign_wsus_cnt("price"))

In [22]:
only_wsus_gcoin_by_user = only_wsus_gcoin.groupBy("account_id") \
    .agg(sum("wsus_cnt").alias("wsus_cnt"), sum("paid_use").alias("paid_use"), sum(col("paid_use") + col("free_use")).alias("total_use"))

In [23]:
only_wsus_gcoin_by_user = only_wsus_gcoin_by_user.withColumn("group", \
                                                when(col("wsus_cnt") <=10, "A") \
                                                .when((col("wsus_cnt") >= 11) & (col("wsus_cnt") <= 50), "B") \
                                                .when((col("wsus_cnt") >= 51) & (col("wsus_cnt") <= 100), "C") \
                                                .when((col("wsus_cnt") >= 101) & (col("wsus_cnt") <= 800), "D") \
                                                .when((col("wsus_cnt") >= 801), "E") \
                                                          )
only_wsus_gcoin_by_user.groupBy("group").agg(countDistinct("account_id").alias("user_cnt"), sum("total_use").alias("total_use"), sum("paid_use").alias("paid_use"), sum("wsus_cnt").alias("wsus_cnt")).orderBy("group").toPandas()

,group,user_cnt,total_use,paid_use,wsus_cnt
0,A,309061,198226800,15399660,1031233
1,B,19399,78403800,58736615,426951
2,C,3934,49978600,43558885,275549
3,D,3456,136607600,121259555,756279
4,E,118,21942800,19629750,121792


In [24]:
only_wsus_gcoin_by_user.toPandas().to_csv("./only_wsus_gcoin_by_user3.csv", index=False)